## 1.ใช้ Morphological Closing เชื่อมส่วนที่แยกกัน
ใช้ cv2.morphologyEx กับ Kernel ขนาดเล็ก เพื่อเชื่อมจุด (dot) เข้ากับฐานตัวอักษร

In [7]:
import cv2
import numpy as np

image = cv2.imread("../../data/test_images/line_detection/test_a_to_z_3.jpg")
if image is None:
    raise FileNotFoundError("ไม่พบไฟล์ภาพ กรุณาตรวจสอบเส้นทางของไฟล์")

# แปลงเป็น Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# การ Threshold แบบ Adaptive Gaussian
binary_image = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51,30
)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))  # ขนาด Kernel
closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

cv2.imshow("test", closed_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## 2.ใช้ระยะห่างระหว่าง Contours เพื่อตัดสินการรวม
หลังการหา Contours ให้ตรวจสอบ Contours ใกล้เคียงกันและรวมพิกัด Bounding Box:

In [13]:
import cv2
import numpy as np

# โหลดภาพ
image = cv2.imread("../../data/test_images/line_detection/test_a_to_z_3.jpg")
if image is None:
    raise FileNotFoundError("ไม่พบไฟล์ภาพ กรุณาตรวจสอบเส้นทางของไฟล์")

# แปลงเป็น Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# การ Threshold แบบ Adaptive Gaussian
binary_image = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 51, 30
)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (6, 6))  # ขนาด Kernel
closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

# หา Contours
contours, _ = cv2.findContours(closed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# เรียง Contours จากซ้ายไปขวา
contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

# รวม Contours ที่มีการทับซ้อนกัน
merged_contours = []
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    should_merge = False
    for i, (mx, my, mw, mh) in enumerate(merged_contours):
        if (x < mx + mw and x + w > mx and  # ทับซ้อนในแนวนอน
            y < my + mh and y + h > my):    # ทับซ้อนในแนวตั้ง
            # รวม Bounding Box
            nx = min(x, mx)
            ny = min(y, my)
            nw = max(x + w, mx + mw) - nx
            nh = max(y + h, my + mh) - ny
            merged_contours[i] = (nx, ny, nw, nh)
            should_merge = True
            break
    if not should_merge:
        merged_contours.append((x, y, w, h))

# ตัดตัวอักษร
char_images = []
for x, y, w, h in merged_contours:
    if w > 5 and h > 5:  # กรองเสียงรบกวน (Bounding Box ขนาดเล็ก)
        char_img = binary_image[y:y+h, x:x+w]
        char_images.append(char_img)

# แสดงภาพผลลัพธ์
output_image = image.copy()
for x, y, w, h in merged_contours:
    cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 1)

cv2.imshow("Detected Characters", output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
